In [1]:
import os
import sys
import types
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sas

import read_clean_quote as rcq
import calculate_trends as ct

In [2]:
n_roll    = 20
n_mom     = 20
n_rsi     = 15
n_max     = 20
days_pred = 5
predictors_df, price_df = rcq.build_quote( 'IBM', n_roll, n_mom, n_rsi, n_max, days_pred )

In [3]:
predictors_df.tail()

,Volatility,Bollinger_20,RSI_15,Momentum_20
Date,,,,
2004-02-06,0.354000,0.678560,0.448133,0.717125
2004-02-05,0.359296,0.681004,0.533333,0.718438
2004-02-04,0.364086,0.796002,0.552753,0.755391
2004-02-03,0.356890,0.815283,0.510990,0.748970
2004-02-02,0.346708,0.805872,0.505729,0.785454


In [4]:
price_df.head()

,Adj Close,Close_mean_20,Close_std_20,Diff_co,Diff_v,Close_1,Close_2,Close_3,Close_4,Close_5
Date,,,,,,,,,,
2017-03-01,181.949997,178.651213,2.990200,0.581450,0.301502,1.184512,1.421406,0.330847,0.165154,0.441625
2017-02-28,179.820007,178.211260,3.123634,0.524529,0.297068,0.234121,-0.843672,-1.007425,-0.734191,-0.244085
2017-02-27,179.399994,177.941303,3.209412,0.401208,0.374564,-1.075275,-1.238646,-0.966050,-0.477089,-0.702941
2017-02-24,181.350006,177.766759,3.220811,0.563259,0.484011,-0.165146,0.110412,0.604688,0.376381,-0.044087
2017-02-23,181.649994,177.562180,3.109244,0.480769,0.248935,0.276014,0.771108,0.542423,0.121259,-0.016512


In [6]:
from sklearn.decomposition import PCA

In [7]:
my_pca = PCA( n_components=4 )
my_pca.fit( predictors_df )
print my_pca.explained_variance_ratio_

[ 0.78222181  0.0987274   0.07546169  0.0435891 ]


In [8]:
# Data should be cleaned, need to plot some trends and check for correlation
# [ 'Diff_co', 'Diff_ao', 'Diff_hl', 'Diff_v'] ]
# [ 'Close_mean_n', 'Close_std_n' ]
# [ 'Momentum_n' ]
#
# Diff co, v, have different info, co same as hl basically
# Momentum 3 correlated with co, 5 a little less, 10 at limit
# HUGE autocorrelation with momentum, long momentum really tracks trends, until at/past scale of momentum

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [10]:
print predictors_df.shape[0]

3293


In [33]:
N_train = 2500
N_test  = 700
off     = days_pred

# Put things in right cronological order
price_df      =      price_df[::-1]
predictors_df = predictors_df[::-1]

# Labels need to be stored as a list, because will use diff CLF for each
train_label_list   = []
test_label_list    = []

# Features are constant for each clf
train_features = predictors_df[:N_train]
test_features  = predictors_df[ N_train+off:N_train+off+N_test]

# Loop over offsets so have training data for each day
for offset in range( 1, off+1 ):
    
    # Train data from 0 to N_train
    # Train labels from ^+offset
    # Test data from N_train+o to N_train + N_test + o
    # Test labels from ^+offset
    
    train_label_list.append(  flipped_df.ix[offset:N_train+offset,'Adj Close'].values  )
    test_label_list.append(   flipped_df.ix[2*offset+N_train::N_test+N_train+2*offset,'Adj Close'].values )

In [30]:
test_features['Adj Close'].head()

Date
2014-01-29    159.390530
2014-01-30    160.257968
2014-01-31    159.643530
2014-02-03    156.228020
2014-02-04    156.173808
Name: Adj Close, dtype: float64

In [32]:
clf_list = []

for i in range( 0, off ):
    
    clf_list.append( SVR() )
    
    clf_list[i].fit( train_features, train_label_list[i] )
    
    print clf_list[i].score( train_features, train_label_list[i] ), test_features.ix[0,'Adj Close'],
    print clf_list[i].predict( test_features.iloc[0] ), test_label_list[i][0]

0.925266725968 159.39053 [ 154.86204236] 162.318117


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.924107025419 159.39053 [ 154.93927547] 159.79715


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.922888925438 159.39053 [ 154.56218893] 160.257968


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.921903704721 159.39053 [ 154.84077323] 156.22802


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


0.91930927933 159.39053 [ 153.77003668] 157.43882


/home/sean/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
